<h1> Optimization electricity consumption </h1>

<h3> Imports </h3>

In [112]:
import math
import random
import copy
import numpy as np
import joblib

%run preprocess_functions.py
from Individual import Individual

<h3> Helper functions </h3>

In [113]:
def calc_consumption_from_planing(plan,mat,day_consumption):
    for house in plan:
        for machine in house.keys():
            for time_slot in house[machine]:
                day_consumption[time_slot] += mat[machine]["power"]
    return day_consumption

<h3> Recuit functions </h3>

In [114]:
#Not Perfect
def initial_solution(empty_plannings,mat):

    for house in range(len(empty_plannings)):
        for machine in empty_plannings[house].keys():
            if type(mat[machine]["nb_time_slot"]) == list:
                mat[machine]["nb_time_slot"] = random.sample(mat[machine]["nb_time_slot"],1)[0]
            
            if machine in ['LL','SL','LV']:
                start_index = random.randint(0,47)
                for i in range(mat[machine]["nb_time_slot"]):
                    empty_plannings[house][machine].append((start_index+i)%48)
            else:
                if mat[machine]["is_sequencable"]:
                    for i in range(mat[machine]["nb_time_slot"]):
                        index = random.randint(0,15)
                        while index in empty_plannings[house][machine]:
                            index = random.randint(0,15)
                else:
                    start_index = random.randint(0,15-mat[machine]["nb_time_slot"])
                    for i in range(mat[machine]["nb_time_slot"]):
                        empty_plannings[house][machine].append(start_index+i)
                

    day_consumption = []
    for i in range(48):
        day_consumption.append(int(0))

    day_consumption = calc_consumption_from_planing(empty_plannings,mat,day_consumption)

    individual = Individual({"start":0,"end":15},empty_plannings,day_consumption)
    

    return individual

In [115]:
def get_neighbour_solution(solution: Individual):
    solution_neigh = copy.deepcopy(solution)
    solution_neigh.mutate()

    return solution_neigh

def cost_function(solution : Individual):
    return solution.getMax()

def metropolis(neighbour_solution, current_solution,temperature):
    return math.exp( -( abs( cost_function(current_solution) - cost_function(neighbour_solution) ) / temperature ) )

def update_temperature(temperature):
    return temperature * 0.99

In [116]:
def recuit_simule(temperature,current_solution):
    best_current_solution = copy.deepcopy(current_solution)
    print_count = 0
    count_iteration_without_improvment = 0

    while count_iteration_without_improvment < 1000:
        print_count += 1
        if print_count%10 == 0:
            print("\r",end="")
            print("**Recuit",print_count,count_iteration_without_improvment,end="                ")

        neighbour_solution = get_neighbour_solution(current_solution)

        if cost_function(neighbour_solution) < cost_function(current_solution) or random.random() < metropolis( neighbour_solution , current_solution , temperature):
            current_solution = neighbour_solution

        if cost_function(current_solution) < cost_function(best_current_solution):
            best_current_solution = copy.deepcopy(current_solution)
            count_iteration_without_improvment = 0
        else:
            count_iteration_without_improvment += 1

        temperature = update_temperature(temperature)
    
    return best_current_solution

<h3> Variables </h3>

In [117]:
matrice_type_puissance_par_demie_heure = {'LV':{"power":float(0.65),"nb_time_slot":2,"is_sequencable":False}, 
                                            'LL':{"power":float(1),"nb_time_slot":2,"is_sequencable":False}, 
                                            'SL':{"power":float(0.125),"nb_time_slot":8,"is_sequencable":False}, 
                                            'TV':{"power":float(0.05),"nb_time_slot":[1,2,3,4,5],"is_sequencable":False}, 
                                            'FG_1':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_1':{"power":float(0.18),"nb_time_slot":12,"is_sequencable":False}, 
                                            'CG':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'FO':{"power":float(0.8),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'PL':{"power":float(0.6),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'FG_2':{"power":float(0.3),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_2':{"power":float(0.25),"nb_time_slot":12,"is_sequencable":False}
                                            }

<h3> Main </h3>

In [118]:
data = load_csv("data/data.csv")
batch_empty_plannings = make_batches("data/data.csv","data/data_network.csv")

In [119]:
solutions = []
nb_batches = len(batch_empty_plannings)
count_batches = 0
for key in batch_empty_plannings.keys():
    batch = batch_empty_plannings[key]

    print("Init Solution")
    current_solution = initial_solution(batch,matrice_type_puissance_par_demie_heure)
    print("Score init :",current_solution.getMax(),"| Batch size :",len(batch)," | Batch number :",str(count_batches)+"/"+str(nb_batches))

    best_found_solution = recuit_simule(1000, current_solution)
    curr_best = cost_function(best_found_solution)

    # print(best_found_solution.day_consumption)
    print("Score optimized",cost_function(best_found_solution))
    solutions.append(best_found_solution)
    count_batches += 1




Init Solution
Score init : 117.35500000000019 | Batch size : 201  | Batch number : 0/98
**Recuit 36580 990                Score optimized 86.18
Init Solution
Score init : 110.08500000000026 | Batch size : 202  | Batch number : 1/98
**Recuit 36110 996                Score optimized 82.18
Init Solution
Score init : 111.16000000000021 | Batch size : 202  | Batch number : 2/98
**Recuit 39430 993                Score optimized 83.48
Init Solution
Score init : 125.23500000000023 | Batch size : 202  | Batch number : 3/98
**Recuit 27020 996                Score optimized 87.82
Init Solution
Score init : 113.66000000000025 | Batch size : 202  | Batch number : 4/98
**Recuit 37460 995                Score optimized 86.08
Init Solution
Score init : 114.61000000000028 | Batch size : 202  | Batch number : 5/98
**Recuit 27980 990                Score optimized 89.88
Init Solution
Score init : 113.83500000000019 | Batch size : 202  | Batch number : 6/98
**Recuit 45550 995                Score optimize

In [120]:
# joblib.dump(best_found_solution.plannings,"out/Plannings.joblib")
# joblib.dump(best_found_solution.day_consumption,"out/Day_Consumption.joblib")
# joblib.dump(best_found_solution,"out/Solution.joblib")
joblib.dump(solutions,"out/Solutions.joblib")

['out/Solutions.joblib']